In [31]:
# Cell 1: Import Libraries and Setup for Data Cleaning
import pandas as pd
import numpy as np
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# File path (same as exploration)
file_path = r'D:\Git repo\real_estate_listings\real_estate_project\data\raw\Real_Estate_data.xlsx'

# Load all sheets
print("📥 Loading data for cleaning...")
sheets = {
    'active': 'Active_Listings',
    'archive': 'Archive', 
    'clients': 'Client_Database'
}

dataframes = {}
for short_name, sheet_name in sheets.items():
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    dataframes[short_name] = df
    print(f"✅ {short_name.title()}: {len(df):,} rows × {df.shape[1]} cols")

print(f"\n🎯 Ready for data cleaning!")
print(f"📊 Total records to process: {sum(len(df) for df in dataframes.values()):,}")


📥 Loading data for cleaning...
✅ Active: 1,000 rows × 28 cols
✅ Archive: 1,000 rows × 16 cols
✅ Clients: 1,000 rows × 7 cols

🎯 Ready for data cleaning!
📊 Total records to process: 3,000


In [33]:
# Cell 2: Standardize Column Names (Create Clean, Consistent Names)
print("🏷️  STANDARDIZING COLUMN NAMES")
print("=" * 40)

def clean_column_name(col_name):
    """Convert column names to clean, lowercase, snake_case format"""
    # Remove special characters and replace with underscore
    clean_name = re.sub(r'[^\w\s]', '_', str(col_name))
    # Replace spaces and multiple underscores with single underscore
    clean_name = re.sub(r'\s+|_+', '_', clean_name)
    # Convert to lowercase and strip leading/trailing underscores
    clean_name = clean_name.lower().strip('_')
    return clean_name

# Store original column names for reference
original_columns = {}

# Clean column names for each dataframe
for name, df in dataframes.items():
    print(f"\n📋 {name.upper()} DATASET:")
    original_columns[name] = df.columns.tolist()
    
    # Create mapping of old to new column names
    column_mapping = {old_col: clean_column_name(old_col) for old_col in df.columns}
    
    # Show the transformation for first 5 columns
    print("   Column name transformations (first 5):")
    for i, (old, new) in enumerate(list(column_mapping.items())[:5]):
        print(f"     {i+1}. '{old}' → '{new}'")
    
    # Apply the column renaming
    dataframes[name] = df.rename(columns=column_mapping)
    
    print(f"   ✅ Renamed {len(column_mapping)} columns")

print(f"\n🎯 Column standardization complete!")
print("📌 All column names are now lowercase, snake_case format")


🏷️  STANDARDIZING COLUMN NAMES

📋 ACTIVE DATASET:
   Column name transformations (first 5):
     1. 'Property ID' → 'property_id'
     2. 'Listing Status' → 'listing_status'
     3. 'Listing Type' → 'listing_type'
     4. 'Listing Date' → 'listing_date'
     5. 'Building / Society' → 'building___society'
   ✅ Renamed 28 columns

📋 ARCHIVE DATASET:
   Column name transformations (first 5):
     1. 'Property ID' → 'property_id'
     2. 'Listing Status' → 'listing_status'
     3. 'Listing Type' → 'listing_type'
     4. 'Listing Date' → 'listing_date'
     5. 'Closing Date' → 'closing_date'
   ✅ Renamed 16 columns

📋 CLIENTS DATASET:
   Column name transformations (first 5):
     1. 'ClientID' → 'clientid'
     2. 'Client Name' → 'client_name'
     3. 'Client Phone' → 'client_phone'
     4. 'Client Email' → 'client_email'
     5. 'Looking For' → 'looking_for'
   ✅ Renamed 7 columns

🎯 Column standardization complete!
📌 All column names are now lowercase, snake_case format


In [34]:
# Cell 3: Clean and Parse Indian Price Columns
print("💰 CLEANING PRICE COLUMNS")
print("=" * 30)

# First, let's see which price columns we have after renaming
print("🔍 Identifying price columns in each dataset:")
for name, df in dataframes.items():
    price_cols = [col for col in df.columns if any(keyword in col.lower() 
                  for keyword in ['price', 'rent', 'deposit', 'amount'])]
    print(f"   {name.title()}: {price_cols}")

# Indian price parsing function
def parse_indian_price(value):
    """
    Parse Indian price formats like '1.25 Cr', '78 Lac', '32,50,000', 'On Request' 
    Returns float in rupees or NaN for non-numeric values
    """
    if pd.isna(value):
        return np.nan
    
    # Convert to string and clean
    price_str = str(value).lower().strip()
    
    # Handle non-numeric cases
    non_price_terms = ['nan', '', 'on request', 'negotiable', 'slightly', 'call for price']
    if price_str in non_price_terms:
        return np.nan
    
    # Remove currency symbols and spaces
    price_str = re.sub(r'[₹\s,]', '', price_str)
    
    # Parse number with units (Cr, Lac, K)
    match = re.match(r'([0-9]*\.?[0-9]+)([a-z]*)', price_str)
    if not match:
        return np.nan
    
    number, unit = match.groups()
    try:
        number = float(number)
    except ValueError:
        return np.nan
    
    # Apply multipliers for Indian units
    multipliers = {
        'cr': 10000000,    # 1 Crore = 10 Million
        'crore': 10000000,
        'lac': 100000,     # 1 Lakh = 100 Thousand  
        'lakh': 100000,
        'l': 100000,       # 'L' often means Lakh
        'k': 1000,         # 1 Thousand
        '': 1              # No unit = rupees
    }
    
    multiplier = multipliers.get(unit, 1)
    return number * multiplier

print(f"\n🧹 Applying price parsing to all price columns...")


💰 CLEANING PRICE COLUMNS
🔍 Identifying price columns in each dataset:
   Active: ['asking_price__â_¹', 'monthly_rent__â_¹', 'security_deposit__â_¹', 'price_negotiable']
   Archive: ['asking_price__â_¹', 'monthly_rent__â_¹', 'final_price__â_¹']
   Clients: []

🧹 Applying price parsing to all price columns...


In [35]:
# Cell 3 (Continued): Apply the price parsing function
# Define the actual price columns (excluding negotiable flag)
price_columns_to_clean = [
    'asking_price__â_¹', 
    'monthly_rent__â_¹', 
    'security_deposit__â_¹',
    'final_price__â_¹'
]

# Apply price parsing to each dataset
for name, df in dataframes.items():
    if name == 'clients':
        print(f"   {name.title()}: No price columns to clean ✅")
        continue
        
    print(f"\n   {name.upper()} DATASET:")
    for col in price_columns_to_clean:
        if col in df.columns:
            # Show sample before cleaning
            sample_before = df[col].dropna().head(3).astype(str).tolist()
            print(f"     🔧 {col}")
            print(f"        Before: {sample_before}")
            
            # Apply price parsing
            before_nulls = df[col].isna().sum()
            df[col] = df[col].apply(parse_indian_price)
            after_nulls = df[col].isna().sum()
            new_nulls = after_nulls - before_nulls
            
            # Show sample after cleaning
            sample_after = df[col].dropna().head(3)
            formatted_sample = [f"₹{x:,.0f}" for x in sample_after] if len(sample_after) > 0 else ["No valid prices"]
            print(f"        After:  {formatted_sample}")
            print(f"        Result: {new_nulls} text values → NaN, {df[col].count()} valid prices")

# Handle price_negotiable separately (it's a boolean flag, not a price)
print(f"\n   🚩 HANDLING NEGOTIABLE FLAGS:")
for name, df in dataframes.items():
    if 'price_negotiable' in df.columns:
        before_values = df['price_negotiable'].unique()[:5]  # Show first 5 unique values
        print(f"     {name.title()} price_negotiable before: {before_values}")
        
        # Convert to boolean (Yes/Slightly/Negotiable = 1, No/other = 0)
        df['price_negotiable'] = df['price_negotiable'].astype(str).str.lower().isin(
            ['yes', 'y', 'slightly', 'negotiable', '1', 'true']
        ).astype(int)
        
        after_values = df['price_negotiable'].unique()
        print(f"     {name.title()} price_negotiable after:  {after_values}")

print(f"\n✅ Price column cleaning complete!")



   ACTIVE DATASET:
     🔧 asking_price__â_¹
        Before: ['45500000.0', '28800000.0', '32000000.0']
        After:  ['₹45,500,000', '₹28,800,000', '₹32,000,000']
        Result: 0 text values → NaN, 601 valid prices
     🔧 monthly_rent__â_¹
        Before: ['45000.0', '170000.0', '85000.0']
        After:  ['₹45,000', '₹170,000', '₹85,000']
        Result: 0 text values → NaN, 399 valid prices
     🔧 security_deposit__â_¹
        Before: ['180000.0', '510000.0', '425000.0']
        After:  ['₹180,000', '₹510,000', '₹425,000']
        Result: 0 text values → NaN, 399 valid prices

   ARCHIVE DATASET:
     🔧 asking_price__â_¹
        Before: ['19600000.0', '10100000.0', '19300000.0']
        After:  ['₹19,600,000', '₹10,100,000', '₹19,300,000']
        Result: 0 text values → NaN, 514 valid prices
     🔧 monthly_rent__â_¹
        Before: ['41000.0', '52000.0', '147000.0']
        After:  ['₹41,000', '₹52,000', '₹147,000']
        Result: 0 text values → NaN, 486 valid prices
     🔧 f

In [36]:
# Cell 4: Clean BHK Format and Other Text Fields
print("🏠 CLEANING BHK AND TEXT FIELDS")
print("=" * 35)

# Function to clean BHK format
def clean_bhk(bhk_value):
    """Convert '3 BHK', '2BHK', 'NaN' to clean integer or NaN"""
    if pd.isna(bhk_value):
        return np.nan
    
    bhk_str = str(bhk_value).upper().strip()
    
    # Extract number from formats like "3 BHK", "2BHK", "3"
    match = re.search(r'(\d+)', bhk_str)
    if match:
        return int(match.group(1))
    else:
        return np.nan

# Clean BHK columns in Active and Archive datasets
print("🔧 Cleaning BHK format:")
for name, df in dataframes.items():
    if 'bedrooms__bhk_' in df.columns:
        col = 'bedrooms__bhk_'  # Standardized column name
        print(f"\n   {name.upper()} - {col}:")
        
        # Show before cleaning
        before_sample = df[col].dropna().head(5).tolist()
        before_nulls = df[col].isna().sum()
        print(f"     Before: {before_sample}")
        print(f"     Null count before: {before_nulls}")
        
        # Apply BHK cleaning
        df[col] = df[col].apply(clean_bhk)
        
        # Show after cleaning
        after_sample = df[col].dropna().head(5).tolist()
        after_nulls = df[col].isna().sum()
        print(f"     After:  {after_sample}")
        print(f"     Null count after: {after_nulls}")
        print(f"     Unique BHK values: {sorted(df[col].dropna().unique())}")

print(f"\n✅ BHK cleaning complete!")


🏠 CLEANING BHK AND TEXT FIELDS
🔧 Cleaning BHK format:

✅ BHK cleaning complete!


In [37]:
# Debug: Check what the actual column names are after standardization
print("🔍 DEBUGGING: Actual column names after standardization")
print("=" * 55)

for name, df in dataframes.items():
    print(f"\n📋 {name.upper()} DATASET columns:")
    
    # Look for BHK-related columns
    bhk_cols = [col for col in df.columns if 'bhk' in col.lower() or 'bedroom' in col.lower()]
    print(f"   BHK-related columns: {bhk_cols}")
    
    # Show all columns (first 10) to see the actual format
    print(f"   First 10 columns: {list(df.columns[:10])}")
    
    # If we find BHK columns, show sample data
    for col in bhk_cols:
        if col in df.columns:
            sample_data = df[col].dropna().head(3).tolist()
            print(f"   Sample {col} data: {sample_data}")


🔍 DEBUGGING: Actual column names after standardization

📋 ACTIVE DATASET columns:
   BHK-related columns: ['bedrooms__bhk']
   First 10 columns: ['property_id', 'listing_status', 'listing_type', 'listing_date', 'building___society', 'area___locality', 'city', 'pincode', 'property_type', 'bedrooms__bhk']
   Sample bedrooms__bhk data: ['3 BHK', '3 BHK', '5 BHK']

📋 ARCHIVE DATASET columns:
   BHK-related columns: ['bedrooms__bhk']
   First 10 columns: ['property_id', 'listing_status', 'listing_type', 'listing_date', 'closing_date', 'building___society', 'area___locality', 'city', 'pincode', 'property_type']
   Sample bedrooms__bhk data: ['3 BHK', '3 BHK', '2 BHK']

📋 CLIENTS DATASET columns:
   BHK-related columns: []
   First 10 columns: ['clientid', 'client_name', 'client_phone', 'client_email', 'looking_for', 'requirements', 'status']


In [38]:
# Cell 4: Clean BHK Format and Other Text Fields (Corrected)
print("🏠 CLEANING BHK AND TEXT FIELDS")
print("=" * 35)

# Function to clean BHK format
def clean_bhk(bhk_value):
    """Convert '3 BHK', '2BHK', 'NaN' to clean integer or NaN"""
    if pd.isna(bhk_value):
        return np.nan
    
    bhk_str = str(bhk_value).upper().strip()
    
    # Extract number from formats like "3 BHK", "2BHK", "3"
    match = re.search(r'(\d+)', bhk_str)
    if match:
        return int(match.group(1))
    else:
        return np.nan

# Clean BHK columns in Active and Archive datasets (using correct column name)
print("🔧 Cleaning BHK format:")
for name, df in dataframes.items():
    if 'bedrooms__bhk' in df.columns:  # Corrected column name
        col = 'bedrooms__bhk'
        print(f"\n   {name.upper()} - {col}:")
        
        # Show before cleaning
        before_sample = df[col].dropna().head(5).tolist()
        before_nulls = df[col].isna().sum()
        print(f"     Before: {before_sample}")
        print(f"     Null count before: {before_nulls}")
        
        # Apply BHK cleaning
        df[col] = df[col].apply(clean_bhk)
        
        # Show after cleaning
        after_sample = df[col].dropna().head(5).tolist()
        after_nulls = df[col].isna().sum()
        print(f"     After:  {after_sample}")
        print(f"     Null count after: {after_nulls}")
        print(f"     Unique BHK values: {sorted(df[col].dropna().unique())}")

print(f"\n✅ BHK cleaning complete!")


🏠 CLEANING BHK AND TEXT FIELDS
🔧 Cleaning BHK format:

   ACTIVE - bedrooms__bhk:
     Before: ['3 BHK', '3 BHK', '5 BHK', '3 BHK', '2 BHK']
     Null count before: 472
     After:  [3.0, 3.0, 5.0, 3.0, 2.0]
     Null count after: 472
     Unique BHK values: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0)]

   ARCHIVE - bedrooms__bhk:
     Before: ['3 BHK', '3 BHK', '2 BHK', '3 BHK', '1 BHK']
     Null count before: 640
     After:  [3.0, 3.0, 2.0, 3.0, 1.0]
     Null count after: 640
     Unique BHK values: [np.float64(1.0), np.float64(2.0), np.float64(3.0)]

✅ BHK cleaning complete!


In [39]:
# Cell 5: Standardize Location and Categorical Text Fields
print("📍 CLEANING LOCATION AND CATEGORICAL FIELDS")
print("=" * 45)

# Function to clean and standardize text fields
def clean_text_field(text_value):
    """Standardize text: proper case, remove extra spaces, handle NaN"""
    if pd.isna(text_value):
        return np.nan
    
    # Convert to string, strip whitespace, and proper case
    clean_text = str(text_value).strip().title()
    
    # Replace multiple spaces with single space
    clean_text = re.sub(r'\s+', ' ', clean_text)
    
    return clean_text if clean_text else np.nan

# Clean key categorical fields
categorical_fields = [
    'city', 'area___locality', 'property_type', 'furnishing', 
    'facing_direction', 'listing_status', 'listing_type'
]

print("🔧 Standardizing categorical fields:")
for name, df in dataframes.items():
    print(f"\n   {name.upper()} DATASET:")
    
    for field in categorical_fields:
        if field in df.columns:
            # Show before cleaning (unique values)
            before_unique = df[field].nunique() if df[field].count() > 0 else 0
            sample_before = df[field].dropna().head(3).tolist()
            
            # Apply text cleaning
            df[field] = df[field].apply(clean_text_field)
            
            # Show after cleaning
            after_unique = df[field].nunique() if df[field].count() > 0 else 0
            sample_after = df[field].dropna().head(3).tolist()
            
            print(f"     📋 {field}:")
            print(f"        Before: {sample_before} ({before_unique} unique)")
            print(f"        After:  {sample_after} ({after_unique} unique)")

print(f"\n✅ Location and categorical field cleaning complete!")



📍 CLEANING LOCATION AND CATEGORICAL FIELDS
🔧 Standardizing categorical fields:

   ACTIVE DATASET:
     📋 city:
        Before: ['Mira Bhayandar', 'Mira Bhayandar', 'Mira Bhayandar'] (1 unique)
        After:  ['Mira Bhayandar', 'Mira Bhayandar', 'Mira Bhayandar'] (1 unique)
     📋 area___locality:
        Before: ['Mira Road East', 'Shivar Garden', 'Shanti Nagar'] (10 unique)
        After:  ['Mira Road East', 'Shivar Garden', 'Shanti Nagar'] (10 unique)
     📋 property_type:
        Before: ['Bungalow', 'Apartment', 'Bungalow'] (4 unique)
        After:  ['Bungalow', 'Apartment', 'Bungalow'] (4 unique)
     📋 furnishing:
        Before: ['Semi-Furnished', 'Semi-Furnished', 'Fully Furnished'] (3 unique)
        After:  ['Semi-Furnished', 'Semi-Furnished', 'Fully Furnished'] (3 unique)
     📋 facing_direction:
        Before: ['West', 'North', 'West'] (6 unique)
        After:  ['West', 'North', 'West'] (6 unique)
     📋 listing_status:
        Before: ['Available', 'Available', 'Avail

In [40]:
# Cell 6: Save Cleaned Data and Generate Cleaning Summary Report
print("💾 SAVING CLEANED DATA AND GENERATING REPORT")
print("=" * 50)

# Create output directory for processed data
import os
output_dir = r'D:\Git repo\real_estate_listings\real_estate_project\data\processed'
os.makedirs(output_dir, exist_ok=True)

# Save cleaned dataframes to CSV and Parquet formats
print("📁 Saving cleaned datasets:")
for name, df in dataframes.items():
    # Save as CSV (human readable)
    csv_path = os.path.join(output_dir, f'{name}_cleaned.csv')
    df.to_csv(csv_path, index=False)
    
    # Save as Parquet (efficient for data processing)
    parquet_path = os.path.join(output_dir, f'{name}_cleaned.parquet')
    df.to_parquet(parquet_path, index=False)
    
    print(f"   ✅ {name.title()}: {len(df):,} rows → CSV & Parquet saved")

# Generate comprehensive cleaning summary
print(f"\n📊 COMPREHENSIVE CLEANING SUMMARY:")
print("=" * 40)

cleaning_summary = {}
for name, df in dataframes.items():
    summary = {
        'dataset': name.title(),
        'total_rows': len(df),
        'total_columns': len(df.columns),
        'memory_usage_mb': round(df.memory_usage(deep=True).sum() / 1024**2, 2),
        'null_percentage': round(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100, 2),
        'complete_columns': (df.isnull().sum() == 0).sum(),
        'data_types': dict(df.dtypes.value_counts())
    }
    cleaning_summary[name] = summary
    
    print(f"\n🏢 {summary['dataset'].upper()} DATASET:")
    print(f"   📏 Dimensions: {summary['total_rows']:,} rows × {summary['total_columns']} columns")
    print(f"   💾 Memory: {summary['memory_usage_mb']} MB")
    print(f"   ✅ Complete columns: {summary['complete_columns']}/{summary['total_columns']}")
    print(f"   📊 Overall completeness: {100-summary['null_percentage']:.1f}%")
    print(f"   🔢 Data types: {summary['data_types']}")

print(f"\n🎯 CLEANING ACHIEVEMENTS:")
print("   ✅ Column names standardized (snake_case format)")
print("   ✅ Price columns parsed (₹ values converted to numeric)")
print("   ✅ BHK format cleaned (text → integers)")
print("   ✅ Categorical fields standardized (proper case)")
print("   ✅ Data types optimized for analysis")
print("   ✅ Files saved in both CSV and Parquet formats")

print(f"\n🚀 READY FOR NEXT PHASE: Feature Engineering!")


💾 SAVING CLEANED DATA AND GENERATING REPORT
📁 Saving cleaned datasets:
   ✅ Active: 1,000 rows → CSV & Parquet saved
   ✅ Archive: 1,000 rows → CSV & Parquet saved
   ✅ Clients: 1,000 rows → CSV & Parquet saved

📊 COMPREHENSIVE CLEANING SUMMARY:

🏢 ACTIVE DATASET:
   📏 Dimensions: 1,000 rows × 28 columns
   💾 Memory: 0.82 MB
   ✅ Complete columns: 24/28
   📊 Overall completeness: 92.6%
   🔢 Data types: {dtype('O'): np.int64(12), dtype('int64'): np.int64(11), dtype('float64'): np.int64(4), dtype('<M8[ns]'): np.int64(1)}

🏢 ARCHIVE DATASET:
   📏 Dimensions: 1,000 rows × 16 columns
   💾 Memory: 0.51 MB
   ✅ Complete columns: 13/16
   📊 Overall completeness: 89.8%
   🔢 Data types: {dtype('O'): np.int64(8), dtype('float64'): np.int64(4), dtype('<M8[ns]'): np.int64(2), dtype('int64'): np.int64(2)}

🏢 CLIENTS DATASET:
   📏 Dimensions: 1,000 rows × 7 columns
   💾 Memory: 0.51 MB
   ✅ Complete columns: 7/7
   📊 Overall completeness: 100.0%
   🔢 Data types: {dtype('O'): np.int64(6), dtype('int64